<a href="https://colab.research.google.com/github/esuda/mineiracao_dados_complexos/blob/master/Big%20Data/Big_Data_010_Trabalho_1_Auxilio_Emergencial_Noriaki_Suda_Wakabayashi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integrantes do Grupo

### - Daniel Noriaki Kurosawa
### - Eric Uyemura Suda
### - Fernando Shigeru Wakabayashi

# Pacote de dados


*   Abra o link do pacote de dados: https://tinyurl.com/bd10-auxilio
*   Selecione a opção "Adicionar Atalho ao Drive"


In [ ]:
!pip install mrjob

     |████████████████████████████████| 439 kB 6.3 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive/auxilio/"

 auxilio_0.1pct.csv.gz
 auxilio_10pct.csv.gz
 auxilio_1pct.csv.gz
 auxilio.csv
 auxilio.csv.gz
'Dicionário de Dados - Auxílio Emergencial - Portal da transparência.pdf'
 estimativa_dou_2021.csv


In [ ]:
!zcat "/content/drive/My Drive/auxilio/auxilio.csv.gz" | head

"MÊS DISPONIBILIZAÇÃO";"UF";"CÓDIGO MUNICÍPIO IBGE";"NOME MUNICÍPIO";"NIS BENEFICIÁRIO";"CPF BENEFICIÁRIO";"NOME BENEFICIÁRIO";"NIS RESPONSÁVEL";"CPF RESPONSÁVEL";"NOME RESPONSÁVEL";"ENQUADRAMENTO";"PARCELA";"OBSERVAÇÃO";"VALOR BENEFÍCIO"
"202011";"";"";"";"00000000000";"***.643.870-**";"";"-2";"";"Não se aplica";"EXTRA CADUN";"7ª";"Não há";"300,00"
"202011";"";"";"";"00000000000";"***.108.816-**";"";"-2";"";"Não se aplica";"CADUN NAO BOLSA";"6ª";"Não há";"300,00"
"202011";"";"";"";"00000000000";"***.095.215-**";"";"-2";"";"Não se aplica";"CADUN NAO BOLSA";"6ª";"Não há";"300,00"
"202011";"";"";"";"00000000000";"***.108.816-**";"";"-2";"";"Não se aplica";"CADUN NAO BOLSA";"7ª";"Não há";"300,00"
"202011";"";"";"";"00000000000";"***.885.226-**";"";"-2";"";"Não se aplica";"CADUN NAO BOLSA";"5ª";"Não há";"600,00"
"202011";"";"";"";"00000000000";"***.885.226-**";"";"-2";"";"Não se aplica";"CADUN NAO BOLSA";"3ª";"Não há";"600,00"
"202011";"";"";"";"00000000000";"***.885.226-**";"";"-2";"";"Nã

In [ ]:
!zcat "/content/drive/My Drive/auxilio/auxilio.csv.gz" | grep '"UF"'

"MÊS DISPONIBILIZAÇÃO";"UF";"CÓDIGO MUNICÍPIO IBGE";"NOME MUNICÍPIO";"NIS BENEFICIÁRIO";"CPF BENEFICIÁRIO";"NOME BENEFICIÁRIO";"NIS RESPONSÁVEL";"CPF RESPONSÁVEL";"NOME RESPONSÁVEL";"ENQUADRAMENTO";"PARCELA";"OBSERVAÇÃO";"VALOR BENEFÍCIO"


In [ ]:
# Validacao do dataset auxiliar
!cat "/content/drive/My Drive/auxilio/estimativa_dou_2021.csv" | head

UF,COD UF,COD MUNIC,NOME DO MUNICÍPIO, POPULAÇÃO ESTIMADA
RO,11,00015,Alta Floresta D'Oeste,22516
RO,11,00023,Ariquemes,111148
RO,11,00031,Cabixi,5067
RO,11,00049,Cacoal,86416
RO,11,00056,Cerejeiras,16088
RO,11,00064,Colorado do Oeste,15213
RO,11,00072,Corumbiara,7052
RO,11,00080,Costa Marques,19255
RO,11,00098,Espigão D'Oeste,33009


In [ ]:
%%file auxilio.py
from mrjob.job import MRJob
from mrjob.protocol import TextProtocol
import re
import logging

class Auxilio(MRJob):

  OUTPUT_PROTOCOL = TextProtocol

  def configure_args(self):
    super(Auxilio, self).configure_args()
    self.add_passthru_arg('--popFile', default="/content/drive/My Drive/auxilio/estimativa_dou_2021.csv")

  def mapper(self, _, line):
    try :
      #---------------
      # Inicio Solucao
      #---------------

      # Retira identificador de texto de aspas duplas
      line = re.sub("[\"]", "", line)

      # Desempacota as variaveis
      anomes_pagamento, uf, cod_ibge, cidade_nome, \
        nis_benef, cpf_benef, nome_benef, \
        nis_resp, cpf_resp, nome_resp, enquadramento, \
        parcela, obs, val_pago = line.split(";")

      # Premissa adotada para dados faltantes, apenas se o cod ibge ou o valor
      #estiver vazio
      if cod_ibge == "":
        raise ValueError("Codigo IBGE vazio")
      elif val_pago == "":
        raise ValueError("Valor pago vazio")

      # Transforma o valor pago em float
      val_pago = float(re.sub("[,]",".",re.sub("[.]", "", val_pago)))

      # Alternativa - Yield para trazer a Uf pelo mapper e nao reducer init
      #yield cod_ibge, (uf, str(val_pago))
      
      #Retorna a chave e o valor pago
      yield cod_ibge, val_pago

    except ValueError as error:
      logging.warning(error.args)
      logging.warning("linha descartada: "+line)
      pass

  def reducer_init(self):
    logging.warning("criando dicionário de municipios")
    popFile = open(self.options.popFile, "r")
    self.popDict = {}
    for line in popFile :
      try :
        uf,coduf,codmun,nome,pop = line.split(',')
        self.popDict[coduf+codmun] = [nome+"-"+uf, int(pop)]
      except:
        pass      
    popFile.close()

  def reducer(self, key, values):
    try :
      #---------------
      # Inicio Solucao
      #---------------
      #---------------
      # Alternativa se fosse para trazer do mapper uma tupla (uf, valor_pago)
      #---------------
      #total_val = 0
      #for value in values:
      #  total_val+=float(value[1])
      ## Como a UF seria a mesma pegar somente a ultima uf do iterador
      #uf = value[0]

      # Trazendo a UF direto do dicionario criado no pre-reducer
      total_val = sum(values)
      city_uf, pop = self.popDict[key]
      yield city_uf, str(total_val/pop)

    except :
      pass

if __name__ == '__main__':
    logging.basicConfig(filename="log.txt")
    Auxilio.run()

Overwriting auxilio.py


In [ ]:
!python auxilio.py "/content/drive/My Drive/auxilio/auxilio_0.1pct.csv.gz" 

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/auxilio.root.20211031.031328.523583
Running step 1 of 1...
job output is in /tmp/auxilio.root.20211031.031328.523583/output
Streaming final output from /tmp/auxilio.root.20211031.031328.523583/output...
Castelo-ES	0.14846137633048792
Colatina-ES	0.1364547041831948
Conceição da Barra-ES	0.11722100447917659
Conceição do Castelo-ES	0.18623418949328782
Divino de São Lourenço-ES	0.141643059490085
Domingos Martins-ES	0.19953106682297772
Dores do Rio Preto-ES	0.1324893272486383
Ecoporanga-ES	0.1473975734130473
Fundão-ES	0.09383797309978105
Governador Lindenberg-ES	0.12462635088526099
Guaçuí-ES	0.168207318628076
Guarapari-ES	0.17996326962584822
Ibatiba-ES	0.19056871683730664
Ibiraçu-ES	0.1417211243209196
Ibitirama-ES	0.16987542468856173
Iconha-ES	0.10651139671944898
Irupi-ES	0.16932416617905208
Itaguaçu-ES	0.18237734229723931
Itapemirim-ES	0.1982721629430443
Itarana-ES	0.143

In [ ]:
!cat log.txt

INFO:mrjob.conf:No configs found; falling back on auto-configuration
INFO:mrjob.runner:Creating temp directory /tmp/auxilio.root.20211031.031328.523583
INFO:mrjob.sim:Running step 1 of 1...
INFO:mrjob.runner:job output is in /tmp/auxilio.root.20211031.031328.523583/output
INFO:mrjob.runner:Streaming final output from /tmp/auxilio.root.20211031.031328.523583/output...
INFO:mrjob.runner:Removing temp directory /tmp/auxilio.root.20211031.031328.523583...


In [ ]:
!rm log.txt

In [ ]:
!python auxilio.py "/content/drive/My Drive/auxilio/auxilio.csv.gz" 

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/auxilio.root.20211031.031404.374283
Running step 1 of 1...
job output is in /tmp/auxilio.root.20211031.031404.374283/output
Streaming final output from /tmp/auxilio.root.20211031.031404.374283/output...
Castelo-ES	142.9592949731268
Colatina-ES	141.10384364716012
Conceição da Barra-ES	124.80437752152228
Conceição do Castelo-ES	153.50345309226353
Divino de São Lourenço-ES	193.82932011331445
Domingos Martins-ES	179.35190504103164
Dores do Rio Preto-ES	175.04843220962755
Ecoporanga-ES	157.31730262001054
Fundão-ES	126.9448590196166
Governador Lindenberg-ES	132.428221046984
Guaçuí-ES	137.70958179268138
Guarapari-ES	169.90434539002678
Ibatiba-ES	181.29758612958673
Ibiraçu-ES	124.38225336587671
Ibitirama-ES	186.6268403171008
Iconha-ES	136.4019740112192
Irupi-ES	178.37134289057929
Itaguaçu-ES	141.7731368902875
Itapemirim-ES	187.2367480046915
Itarana-ES	163.35301447330585
Iúna

In [ ]:
drive.flush_and_unmount()